# Bootstrapping

How do we replicate getting a dataset again and again?

Bootstrapping is *sampling with replacement* from a dataset randomly n times to create a new dataset.

Any parameter estimated past on this **bootstrap sample** is called a **bootstrap replicate**.

In [3]:
import numpy as np

In [8]:
# create a bootstrap sample

dataset = np.array([1,2,3,4,5])

bs_sample = np.random.choice(dataset, size = 5)

In [10]:
# calculate a bootstrap replicate

print('The bootstrap replicate mean is:', np.mean(bs_sample))

print('The bootstrap replicate median is:', np.median(bs_sample))

print('The bootstrap replicate standard deviation is:', np.std(bs_sample))

The bootstrap replicate mean is: 2.4
The bootstrap replicate median is: 2.0
The bootstrap replicate standard deviation is: 1.4966629547095764


In [ ]:
# given dataset called rainfall; annual rainfall at Sheffield Weather Station from 1883 to 2015

# create 50 bootstrap samples and plot the cdfs
for _ in range(50):
    # Generate bootstrap sample: bs_sample
    bs_sample = np.random.choice(rainfall, size=len(rainfall))

    # Compute and plot ECDF from bootstrap sample
    x, y = ecdf(bs_sample)
    _ = plt.plot(x, y, marker='.', linestyle='none',
                 color='gray', alpha=0.1)

# Compute and plot ECDF from original data
x, y = ecdf(rainfall)
_ = plt.plot(x,y, marker='.')

# Make margins and label axes
plt.margins(0.02)
_ = plt.xlabel('yearly rainfall (mm)')
_ = plt.ylabel('ECDF')

# Show the plot
plt.show()

## Probabilistic estimates

In [ ]:
# given dataset called rainfall; annual rainfall at Sheffield Weather Station from 1883 to 2015

# Take 10,000 bootstrap replicates of the mean: bs_replicates; see dependencies
bs_replicates = draw_bs_reps(rainfall, np.mean, size = 10000)

# The standard deviation of this distribution is called STANDARD ERROR OF THE MEAN
# Compute and print SEM
sem = np.std(rainfall) / np.sqrt(len(rainfall))
print(sem)

# Compute and print standard deviation of bootstrap replicates
# This will be the same as calculating the SEM
bs_std = np.std(bs_replicates)
print(bs_std)

# Make a histogram of the results
_ = plt.hist(bs_replicates, bins=50, normed=True)
_ = plt.xlabel('mean annual rainfall (mm)')
_ = plt.ylabel('PDF')

# Show the plot
plt.show()

## Confidence Intervals

In [ ]:
np.percentile(bs_replicates, [2.5, 97.5])  # 95% confidence interval

## Pairs Bootstraps

When you cannot resample individual data because each sample consists of two variables (e.g., total votes, and vote share for Obama) we need to resample the pairs.

Before we were able to use np.random.choice() to sample the dataset. But it can only do so from a 1D array. Instead we will sample the indices and use these to slice the datasets.

In [11]:
# create an array of sequential integers
np.arange(7)

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
# create an array of sequential integers length of dataset
inds = np.arange(len(total_votes))

# create a bootstrap sample of indices
bs_inds = np.random.choice(inds, len(inds))

# slice out the respective values from the original data arrays
bs_total_votes = total_votes[bs_inds]
bs_dem_share = dem_share[bs_inds]

In [ ]:
# compute a pairs bootstrap replicate
bs_slope, bs_intercept = np.polyfit(bs_total_votes, bs_dem_share, 1)

Use defined method and plot results

In [ ]:
# Generate replicates of slope and intercept using pairs bootstrap
bs_slope_reps, bs_intercept_reps = draw_bs_pairs_linreg(illiteracy, fertility, size = 1000)

# Compute and print 95% CI for slope
print(np.percentile(bs_slope_reps, [2.5, 97.5]))

# Plot the histogram
_ = plt.hist(bs_slope_reps, bins=50, normed=True)
_ = plt.xlabel('slope')
_ = plt.ylabel('PDF')
plt.show()

In [ ]:
# Generate array of x-values for bootstrap lines: x
x = np.array([0,100])

# Plot the bootstrap lines
for i in range(100):
    _ = plt.plot(x, 
                 bs_slope_reps[i]*x + bs_intercept_reps[i],
                 linewidth=0.5, alpha=0.2, color='red')

# Plot the data
_ = plt.plot(illiteracy, fertility, marker='.', linestyle = 'none')

# Label axes, set the margins, and show the plot
_ = plt.xlabel('illiteracy')
_ = plt.ylabel('fertility')
plt.margins(0.02)
plt.show()

## Dependencies

In [ ]:
def bootstrap_replicate_1d(data, func):
    """
    Generate bootstrap replicate of 1D data.
    """
    
    bs_sample = np.random.choice(data, len(data))
    
    return func(bs_sample)

In [ ]:
def draw_bs_reps(data, func, size=1):
    """
    Draw bootstrap replicates.
    """

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

In [ ]:
def draw_bs_pairs_linreg(x, y, size=1):
    """
    Perform pairs bootstrap for linear regression.
    """

    # Set up array of indices to sample from: inds
    inds = np.arange(len(x))

    # Initialize replicates: bs_slope_reps, bs_intercept_reps
    bs_slope_reps = np.empty(size)
    bs_intercept_reps = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_inds = np.random.choice(inds, size=len(inds))
        bs_x, bs_y = x[bs_inds], y[bs_inds]
        bs_slope_reps[i], bs_intercept_reps[i] = np.polyfit(bs_x, bs_y, 1)

    return bs_slope_reps, bs_intercept_reps